# Анализ базы данных по книгам (выпускной проект по SQL)

**Описание**  
Коронавирус застал мир врасплох, изменив привычный порядок вещей. На какое-то время жители городов перестали выходить на улицу, посещать кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.  
Наша компания решила быть на волне и купила крупный сервис для чтения книг по подписке.  
Наша первая задача как аналитика — проанализировать базу данных. В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.  
  
**Задания:**  
Посчитать, сколько книг вышло после 1 января 2000 года;  
Для каждой книги посчитать количество обзоров и среднюю оценку;  
Определить издательство, которое выпустило наибольшее число книг толще 50 страниц, исключив из анализа брошюры;  
Определить автора с самой высокой средней оценкой книг, учитывая только книги с 50 и более оценками;  
Посчитать среднее количество обзоров от пользователей, которые поставили больше 48 оценок.  
  
Подключимся к базе данных:

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

# чтобы выполнить SQL-запрос, используем Pandas
query = '''SELECT * FROM books LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [2]:
query = '''SELECT * FROM authors LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [3]:
query = '''SELECT * FROM publishers LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [4]:
query = '''SELECT * FROM ratings LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [5]:
query = '''SELECT * FROM reviews LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


**Описание данных**

Таблица books  
Содержит данные о книгах:  
book_id — идентификатор книги;  
author_id — идентификатор автора;  
title — название книги;  
num_pages — количество страниц;  
publication_date — дата публикации книги;  
publisher_id — идентификатор издателя.  
  
Таблица authors  
Содержит данные об авторах:  
author_id — идентификатор автора;  
author — имя автора.  
  
Таблица publishers  
Содержит данные об издательствах:  
publisher_id — идентификатор издательства;  
publisher — название издательства;  
  
Таблица ratings  
Содержит данные о пользовательских оценках книг:  
rating_id — идентификатор оценки;  
book_id — идентификатор книги;  
username — имя пользователя, оставившего оценку;  
rating — оценка книги.  
  
Таблица reviews  
Содержит данные о пользовательских обзорах:  
review_id — идентификатор обзора;  
book_id — идентификатор книги;  
username — имя автора обзора;  
text — текст обзора.

## задание - посчитать, сколько книг вышло после 1 января 2000 года

In [6]:
query = '''

SELECT COUNT(book_id) FROM books WHERE (publication_date>'2000-01-01')

'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,819


После 1 января 2000 года вышло 819 книг

## задание - для каждой книги посчитать количество обзоров и среднюю оценку

In [7]:
query = '''

SELECT b.book_id, b.title,
       COUNT(DISTINCT(r.review_id)) AS review_count,
       ROUND(AVG(rt.rating),2) AS average_raiting
FROM books as b        

LEFT JOIN reviews as r
ON b.book_id = r.book_id

LEFT JOIN ratings as rt
ON b.book_id = rt.book_id

GROUP BY b.book_id
ORDER BY review_count DESC, average_raiting DESC

'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,title,review_count,average_raiting
0,948,Twilight (Twilight #1),7,3.66
1,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.41
2,299,Harry Potter and the Chamber of Secrets (Harry...,6,4.29
3,656,The Book Thief,6,4.26
4,734,The Glass Castle,6,4.21
...,...,...,...,...
995,191,Disney's Beauty and the Beast (A Little Golden...,0,4.00
996,387,Leonardo's Notebooks,0,4.00
997,221,Essential Tales and Poems,0,4.00
998,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.67


Вывели количество обзоров и среднюю оценку по каждой книге, отсортировав список сначала по количеству обзоров, затем по средней оценке. Всего в базе 1000 книг. Максимальное количество отззывов 7, минимальное - 0. Средние оценки у всех книг разные.

## задание - определить издательство, которое выпустило наибольшее число книг толще 50 страниц, исключив из анализа брошюры

In [8]:
query = '''

SELECT p.publisher

FROM books as b LEFT JOIN publishers as p
ON b.publisher_id = p.publisher_id

WHERE b.publisher_id in (       -- найдём id такого издательства
        SELECT publisher_id
        FROM books 
        WHERE num_pages>50
        GROUP BY publisher_id 
        ORDER BY COUNT(book_id) DESC
        LIMIT 1 )

LIMIT 1

'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher
0,Penguin Books


Наибольшее число книг (толще 50 страниц) выпустило издательство Penguin Books.

## задание - определить автора с самой высокой средней оценкой книг, учитывая только книги с 50 и более оценками

In [10]:
query = '''

WITH book50 AS (
        -- отбираем книги с 50 и более оценками
                SELECT book_id
                FROM ratings
                GROUP BY book_id
                HAVING COUNT(*) > 50 )

SELECT a.author

FROM books as b JOIN book50 as b50
ON b.book_id = b50.book_id

JOIN authors as a
ON b.author_id = a.author_id

JOIN ratings as rt
ON b50.book_id = rt.book_id

GROUP BY a.author
ORDER BY AVG(rt.rating) DESC

LIMIT 1

'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,author
0,J.K. Rowling/Mary GrandPré


Самая высокой средняя оценка книг, учитывая только книги с 50 и более оценками, у пары авторов J.K. Rowling и Mary GrandPré.

## задание - посчитать среднее количество обзоров от пользователей, которые поставили больше 48 оценок

In [11]:
query = '''

SELECT AVG(review_count) FROM (

    SELECT username, COUNT(DISTINCT(review_id)) AS review_count     -- посчитаем количество обзоров 
    FROM reviews                                                    -- активных пользователей

    WHERE username in (         -- отберём пользователей, которые поставили больше 48 оценок
            SELECT username 
            FROM ratings        
            GROUP BY username
            HAVING (COUNT(*)>48) )

    GROUP BY username
    ORDER BY review_count DESC ) AS users

'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,avg
0,24.0


В среднем пользователи, которые поставили больше 48 оценок, оставили по 24 обзора

# Итоговый вывод

В данной работе мы проанализировали базу данных с информацией о книгах, издательствах, авторах, а также пользовательские обзоры книг, выполнив следующие задания:  
Посчитали, сколько книг вышло после 1 января 2000 года - 819;  
Для каждой книги посчитали количество обзоров и среднюю оценку;  
Определили издательство, которое выпустило наибольшее число книг толще 50 страниц, исключив из анализа брошюры - Penguin Books;  
Определили автора с самой высокой средней оценкой книг, учитывая только книги с 50 и более оценками, - J.K. Rowling/Mary GrandPré;  
Посчитали среднее количество обзоров от пользователей, которые поставили больше 48 оценок, - 24.  
